In [1]:
      import ipyleaflet 
      from ipyleaflet import Map      
      import json
      import pandas as pd
      import os
      import requests
      import geopandas as gpd
      from ipywidgets import link, FloatSlider
      from branca.colormap import linear

      def load_data(url, filename, file_type):
          r = requests.get(url)
          with open(filename, 'w') as f:
              f.write(r.content.decode("utf-8"))
          with open(filename, 'r') as f:
              return file_type(f)
            
      filename = 'COVID19_Cases_US.shp'
      data = gpd.read_file(filename)
      
      #cleaning data to match total # of states to json file
      data=data[data['Province_S'] != 'District of Columbia']
      data=data[data['Province_S'] != 'Northern Marinara Islands']      
      

In [2]:
    geo_json_data = load_data('https://raw.githubusercontent.com/jupyter-widgets/ipyleaflet/master/examples/us-states.json','us-states.json',json.load)
    
    group_by_state = data.loc[:, ['Province_S','Confirmed']].groupby('Province_S')
    group_by_state.head()
    group_by_state_sum = group_by_state.sum()
    
    
    state_id = []
    for i in range(len(geo_json_data['features'])):
             state_id.append(geo_json_data['features'][i]['id'])
            
    corona_choro_data =  dict(zip(state_id, group_by_state_sum['Confirmed'].tolist()))


    layer = ipyleaflet.Choropleth(
          geo_data=geo_json_data,
          choro_data=corona_choro_data,
          colormap=linear.YlOrRd_04,
          border_color='black',
          style={'fillOpacity': 0.8, 'dashArray': '5, 5'})

    m = ipyleaflet.Map(center = (43,-100), zoom = 4)
    m.add_layer(layer)
    m  


Map(center=[43, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…